In [1]:
from ultralytics import YOLO
#from ultralytics.yolo.v8.detect.predict import DetectionPredictor
import cv2

In [ ]:
model = YOLO("yolov8m.pt")


results = model.predict(source="0" , show = True)
cap = source

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # If frame is read correctly, ret is True
    if not ret:
        print("Error: Can't receive frame (stream end?). Exiting ...")
        break

    # Display the resulting frame
    cv2.imshow('Live Video Feed', frame)
    
    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


1/1: 0... Success ✅ (inf frames of shape 640x480 at 30.00 FPS)


WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

0: 480x640 1 person, 2903.6ms
0: 480x640 1 person, 4032.8ms
0: 480x640 1 person, 2488.9ms
0: 480x640 1 person, 1932.5ms
0: 480x640 1 person, 1637.9ms
0: 480x640 1 person, 1793.6ms
0: 480x640 1 person, 1 cell phone, 1851.5ms
0: 480x640 1 person, 1 cell phone, 1948.2ms
0: 480x640 1 person, 2 cell phones, 1 toothbrush, 1926.8ms
0: 480x640 1 person, 1 cell phone, 1 toothbrush, 1973.3ms
0: 480x

In [ ]:
import json
import requests

# Run inference on an image
url = "https://api.ultralytics.com/v1/predict/7KoQv76HZPjBdcbcTLiC"
headers = {"x-api-key": "dcf23897823a23232ac0e2f5599d1108264f5aefab"}
data = {"size": 640, "confidence": 0.25, "iou": 0.25}


with open("path/to/image.jpg", "rb") as f:
	response = requests.post(url, headers=headers, data=data, files={"image": f})

# Check for successful response
response.raise_for_status()

# Print inference results
print(json.dumps(response.json(), indent=2))

In [3]:
import cv2
import requests
import time

def capture_frame():
    # Initialize the webcam
    cap = cv2.VideoCapture(0)

    # Check if the webcam is opened correctly
    if not cap.isOpened():
        raise IOError("Cannot open webcam")

    ret, frame = cap.read()
    cap.release()

    if not ret:
        print("Failed to capture frame from webcam")
        return None
    return frame

def encode_frame(frame):
    # Encode the frame in JPEG format
    _, buffer = cv2.imencode('.jpg', frame)
    
    # Convert the buffer to a byte stream
    return buffer.tobytes()

def send_frame(img_bytes, attempt=0):
    url = "https://api.ultralytics.com/v1/predict/7KoQv76HZPjBdcbcTLiC"
    headers = {"x-api-key": "dcf23897823a23232ac0e2f5599d1108264f5aefab"}
    data = {"size": 640, "confidence": 0.25, "iou": 0.45}
    
    try:
        # Send the POST request with the image
        response = requests.post(url, headers=headers, data=data, files={'image': ('image.jpg', img_bytes, 'image/jpeg')})
        
        if response.status_code == 200:
            print("Success:", response.json())
        else:
            print("Failed:", response.content)
            raise Exception("Server returned non-200 status code")

    except Exception as e:
        print("Request failed:", e)
        if attempt < 3:  # Retry up to 3 times
            time.sleep(2 ** attempt)  # Exponential backoff
            send_frame(img_bytes, attempt + 1)
        else:
            print("Giving up after 3 attempts.")

def main():
    frame = capture_frame()
    if frame is not None:
        img_bytes = encode_frame(frame)
        send_frame(img_bytes)

if __name__ == "__main__":
    main()


Failed: b'{"message":"Unhandled server error.","success":false}\n'
Request failed: Server returned non-200 status code
Failed: b'{"message":"Unhandled server error.","success":false}\n'
Request failed: Server returned non-200 status code
Failed: b'{"message":"Unhandled server error.","success":false}\n'
Request failed: Server returned non-200 status code
Failed: b'{"message":"Unhandled server error.","success":false}\n'
Request failed: Server returned non-200 status code
Giving up after 3 attempts.


In [7]:
pip install JSON

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement JSON (from versions: none)
ERROR: No matching distribution found for JSON


In [5]:
import cv2
import requests
import json
import time

API_URL = "https://api.ultralytics.com/v1/predict/7KoQv76HZPjBdcbcTLiC"
HEADERS = {
    "x-api-key": "dcf23897823a23232ac0e2f5599d1108264f5aefab",
}

def send_frame_for_detection(image_bytes):
    files = {'image': ('image.jpg', image_bytes, 'image/jpeg')}
    response = requests.post(API_URL, headers=HEADERS, files=files)
    if response.status_code == 429:  # Assuming 429 is the rate limit status code
        raise Exception("Rate limit exceeded")
    return response.json()

def resize_frame(frame, target_width=640):
    height, width = frame.shape[:2]
    scale_ratio = target_width / width
    target_height = int(height * scale_ratio)
    resized_frame = cv2.resize(frame, (target_width, target_height), interpolation=cv2.INTER_AREA)
    return resized_frame

def main():
    cap = cv2.VideoCapture(0)
    delay = 1.0  # Initial delay in seconds

    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Error: Can't receive frame (stream end?). Exiting ...")
                break

            resized_frame = resize_frame(frame, target_width=640)
            _, buffer = cv2.imencode('.jpg', resized_frame)
            img_bytes = buffer.tobytes()

            try:
                # Send frame for object detection
                detection_results = send_frame_for_detection(img_bytes)
                print(detection_results)
                delay = max(1.0, delay * 0.9)  # Decrease delay after a successful request
            except Exception as e:
                print("Caught exception:", str(e))
                delay = delay * 2  # Increase delay
                print("Increasing delay to:", delay)

            time.sleep(delay)  # Dynamic delay

            cv2.imshow('Live Object Detection', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


{'message': 'Unhandled server error.', 'success': False}
{'message': 'Unhandled server error.', 'success': False}
{'message': 'Unhandled server error.', 'success': False}
{'message': 'Unhandled server error.', 'success': False}
{'message': 'Unhandled server error.', 'success': False}
{'message': 'Unhandled server error.', 'success': False}
